## Linear models with Polars-ols

The Polars-ols plugin allows you to fit linear models using Polars expressions. You can install it (along with the `patsy` package for formulae) here

### What are linear models?
If you aren't familiar with linear models these are the classic fit-a-straight-line through the data. In a linear model we are trying to predict a target variable (referred to as `y` below) with one or more predictor variables (the `x`s below).

Here is one (of many) simple introductions to linear models on youtube: https://www.youtube.com/watch?v=CtsRRUddV2s


We begin by importing polars and polars_ols.



In [ ]:
import polars as pl
import polars_ols as pls  
import numpy as np

Polars-ols is a Polars *plugin*. When we import a plugin the plugin registers its *namespace* with Polars. A namespace is a set of expressions that are gathered under a title. We have already met built-in namespaces such as `dt` for timeseries expressions or `str` for string expressions

We create a `DataFrame` with a some predictor columns `x1` and `x2`(the values of these columns are arbitrary). We create the target variable `y` as a linear sum of these columns with some random noise

In [ ]:
df = (
    pl.DataFrame(
        {
            "x1": [0.72, -2.43, -0.63, 0.05, -0.07, 0.65, -0.02, -1.64, -0.92, -0.27],
            "x2": [0.24, 0.18, -0.95, 0.23, 0.44, 1.01, -2.08, -1.36, 0.01, 0.75],
        }
    )
    .with_columns(
        # Construct the target variable y with known coefficients of x1 and x2
        # The linear model aims to find these coefficients
        y = 2*pl.col("x1") + 3*pl.col("x2") + np.random.standard_normal(10)
    )
)
df.head()

We can do a scatter plot showing the relationship of the target variable to a predictor variable

In [ ]:
(
    df
    .plot
    .scatter(
        x="x1",
        y="y"
    )
)

We start by fitting an ordinary least squares (i.e. vanilla linear regression) model. We specify:
- the target column as `pl.col("y")`
- an ordinary least squares model with the `least_squares.ols` expression
- the predictors as a list of expressions inside `least_squares.ols`
- the name of the output column of predictions with `alias`

In [ ]:
ols_expr = (
    pl.col("y")
    .least_squares.ols(
        pl.col("x1"), pl.col("x2")
    )
    .alias("ols")
)

We can then add a column with the predictions by passing the expression to `with_columns`

In [ ]:
(
    df
    .with_columns(
        ols_expr
    )
)       

### Coefficients
If we want the regression coefficients instead of the predictions we can set the `mode` of the expression

In [ ]:
ols_coeff_expr = (
    pl.col("y")
    .least_squares.ols(
        pl.col("x1"), 
        pl.col("x2"),
        mode="coefficients",
        add_intercept=True
    )
    .alias("ols_intercept")
)

We then get the coefficients as a `pl.Struct` column

In [ ]:
(
    df
    .select(
        ols_coeff_expr
    )
)       

The order here is `x1`, `x2`,`intercept`. We can get the variable names if we `unnest` the struct

In [ ]:
print(
    df
    .select(
        ols_coeff_expr
    )
    .unnest("ols_intercept")
)       

Despite the noise and small dataset a linear model has done a decent job of finding coefficients for `x1` and `x2` that are close to those we specified when creating `y` above.

### Regularised regression

For practical applications of linear regression we often want to apply regularisation to [damp the effect of noisy data](https://builtin.com/data-science/overfitting-regularization-python).

We can do that in polars-ols with:
- Lasso regression (that uses an L1 norm for the regularisation)
- Ridge regression(that uses an L2 norm for the regularisation)
- Elastic regression (that uses both L1 and L2 norms for the regularisation)

In [ ]:
lasso_expr = pl.col("y").least_squares.lasso(pl.col("x1"), pl.col("x2"), alpha=0.0001, add_intercept=True)
ridge_expr = pl.col("y").least_squares.ridge(pl.col("x1"), pl.col("x2"), alpha=0.0001, add_intercept=True)
elastic_expr = pl.col("y").least_squares.elastic_net(pl.col("x1"), pl.col("x2"), alpha=0.0001,l1_ratio=0.5, add_intercept=True)

See the [Scikit-learn docs](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) for the linear models with the same names for more background on the modelling method

We now make predictions with these models

In [ ]:
(
    df
    .with_columns(
        lasso_expr.round(3).alias("predictions_lasso"),
        ridge_expr.round(3).alias("predictions_ridge"),
        elastic_expr.round(3).alias("predictions_elastic"),
    )
)


> I've compared the results of the polars-ols Elastic Net model with the results from the Scikit-learn library in my production pipelines and they closely match.


## Fitting models by groups
We may want to fit a different model for different subgroups of the data. First we make a new `DataFrame` with a `groups` column

In [ ]:
df_groups = pl.DataFrame(
    {
        "y": [1.16, -2.16, -1.57, 0.21, 0.22, 1.6, -2.11, -2.92, -0.86, 0.47],
        "x1": [0.72, -2.43, -0.63, 0.05, -0.07, 0.65, -0.02, -1.64, -0.92, -0.27],
        "x2": [0.24, 0.18, -0.95, 0.23, 0.44, 1.01, -2.08, -1.36, 0.01, 0.75],
        "groups":[0]*5 + [1]*5
    }
)
df_groups.head()

We can then fit a separate linear model for each group using `over`

In [ ]:
ols_groups_expr = (
    pl.col("y")
    .least_squares.ols(
        pl.col("x1"), 
        pl.col("x2")
    )
    .over("groups")
    .alias("ols")
)

We can again add a column of predictions by calling this expression in `with_columns`

In [ ]:
(
    df_groups
    .with_columns(
        ols_groups_expr
    )
)       

## Making predictions on new data
In the examples above we made predictions on the same data that we used to train the model. We see here how we can fit a model on one set of data and make predictions on another.

First we need to fit a model to get the coefficients. We use the basic `ols` model again with `mode="coefficients"`

In [ ]:
ols_coef_expr = (
    pl.col("y")
    .least_squares.ols(pl.col("x1"), pl.col("x2"), mode="coefficients",add_intercept=True)
    .alias("coef")
)


We can use this to make a `DataFrame` of variable column names and coefficients

In [ ]:
(
    df
    .select(
        ols_coef_expr
    )
    .unnest("coef")
    .unpivot()
)

Now we can make a class to fit the model and then make predictions on new data. The general flow is:
- initialise the class with the model fit expression
- fit the model in `fit` to get the coefficients
- make predictions by:
    - adding a row index to keep track of which row data came from
    - piping the output to a function that joins the predictions
    - in the join select the predictor columns along with the row index
    - `unpivot` the predictor columns
    - join the coefficients
    - multiply the predictors by the coefficients
    - `group_by` to gather the predictors back up into rows
    - `agg` to get the sum of the predictors for the total prediction for each row

In [ ]:
from typing import List

class LinearRegressor:
    def __init__(
        self,
        target_col:str="y",
        feature_cols:List[str]=["x1","x2"],
        model="ols",
        add_intercept:bool=False
    ):
        self.target_col = target_col
        self.feature_cols = [pl.col(feature) for feature in feature_cols]
        self.add_intercept = add_intercept
        if model == "ols":
            self.model_expr = (
            pl.col(self.target_col)
            .least_squares.ols(
                *self.feature_cols,
                mode="coefficients",
                add_intercept=self.add_intercept
            )
            .alias("coef")
        )

    def fit(self, X):
        # Fit the model and save the coefficients in a DataFrame
        self.coefficients_df = (
            X
            .select(
                self.model_expr
            )
            .unnest("coef")
        )
        self.coef_ = (
            self.coefficients_df
            .select(self.feature_cols)
            .unpivot()
        )
        if self.add_intercept:
            self.intercept_ = self.coefficients_df["const"][0]
        else:
            self.intercept_ = 0.0
        return self

    def transform(self, X: pl.DataFrame):
        # Make predictions using the saved coefficients
        return (
            X
            # Add a row index
            .with_row_index()
            .pipe(
                # Join the predictions
                lambda X: X.join(
                    # Select the predictor columns
                    X.select("index", *self.feature_cols)
                    # Unpivot (so we can join the coefficients)
                    .unpivot(index="index",value_name="predictor")
                    .join(
                        # Join the coefficients
                    (
                        X
                        .select(
                            self.model_expr
                        )
                        .unnest("coef")
                        .unpivot(value_name="coef")
                    ),
                        on="variable",
                    )
                    # Multiply by the predictors
                    .with_columns(pred=(pl.col("predictor") * pl.col("coef")))
                    # Gather back up into rows
                    .group_by("index")
                    .agg(
                        pl.col("pred").sum() + self.intercept_
                    ),
                    on="index",
                )
            )
            .sort("index")
        )


Now we make train and test `DataFrames`

In [ ]:
df_train = df[:7]
df_test = df[7:]

We then: 
- instantiate the model
- `fit` the model on `df_train`
- make predictions on `df_test`

In [ ]:
linear_regressor = LinearRegressor(target_col="y",feature_cols=["x1","x2"],model="ols")
linear_regressor.fit(X=df_train)
linear_regressor.transform(X=df_test)

See the repo page for more: https://github.com/azmyrajab/polars_ols/

## Exercises
We read the `load` and weather data

In [ ]:
grid_load_csv = "../data/grid_load.csv"
weather_csv = "../data/weather.csv"

In [ ]:
df_load = pl.read_csv(grid_load_csv,try_parse_dates=True)
df_load.head()

In [ ]:
df_weather = pl.read_csv(weather_csv,try_parse_dates=True)
df_weather.head(2)

- Join the weather data to the load data
- Add a new feature with the weekday from the date

In [ ]:
df_grid = (
    <blank>
)
df_grid

Take data before 2020 as the training data and data in 2020 as the test data

In [ ]:
from datetime import date
df_train = (
    df_grid
    .filter(pl.col("time")<date(2019,4,1))
)
df_test = (
    df_grid
    .filter(pl.col("time")>=date(2019,4,1))
)

We make a list of feature columns

In [ ]:
features = df_grid.columns[2:]
features

Fit the `LinearRegressor` class with an intercept

In [ ]:
linear_regressor = LinearRegressor(
    target_col="load",
    feature_cols=features,
    model="ols",
    add_intercept=True
)
linear_regressor.fit(X = df_train)
pred_df = linear_regressor.transform(X = df_test)
pred_df

Plot the `load` and the `pred` as line charts with `plot.line`

In [ ]:
#Hint
# `unpivot` the load and pred columns with time as the index column

## Solutions

In [ ]:
grid_load_csv = "../data/grid_load.csv"
weather_csv = "../data/weather.csv"

In [ ]:
df_load = pl.read_csv(grid_load_csv,try_parse_dates=True)
df_load.head()

In [ ]:
df_weather = pl.read_csv(weather_csv,try_parse_dates=True)
df_weather.head(2)

- Join the weather data to the load data
- Add a new feature with the weekday from the date

In [ ]:
df_grid = (
    pl.read_csv(grid_load_csv ,try_parse_dates=True)
    .join(
        pl.read_csv(weather_csv,try_parse_dates=True),
        on="time",
        how="inner"
    )
    .with_columns(
        pl.col("time").dt.weekday().alias("weekday"),
        pl.selectors.float().fill_null(strategy="mean")
    )
)
df_grid

Take data before 2020 as the training data and data in 2020 as the test data

In [ ]:
from datetime import date
df_train = (
    df_grid
    .filter(pl.col("time")<date(2019,4,1))
)
df_test = (
    df_grid
    .filter(pl.col("time")>=date(2019,4,1))
)

In [ ]:
features = df_grid.columns[2:]
features

Fit the `LinearRegressor` class with an intercept

In [ ]:
linear_regressor = LinearRegressor(
    target_col="load",
    feature_cols=features,
    model="ols",
    add_intercept=True
)
linear_regressor.fit(X = df_train)
pred_df = linear_regressor.transform(X = df_test)
pred_df

Plot the `load` and the `pred` time series

In [ ]:
(
    pred_df
    .unpivot(
        index="time",
        on=["load","pred"]
    )
    .plot
    .line(
        x="time",
        y="value",
        color="variable"
    )
)